In [20]:
import numpy as np

import torch
import torch.nn as nn

from torchvision import datasets, models, transforms

In [21]:
model = models.resnet18(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [22]:
resnet_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# resnet_transforms = models.ResNet18_Weights.IMAGENET1K_V1.transforms()

In [23]:
train_data = datasets.ImageFolder('/kaggle/input/70-dog-breedsimage-data-set/train', transform=resnet_transforms)
val_data = datasets.ImageFolder('/kaggle/input/70-dog-breedsimage-data-set/valid', transform=resnet_transforms)
test_data = datasets.ImageFolder('/kaggle/input/70-dog-breedsimage-data-set/test', transform=resnet_transforms)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)
test_laoder = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

In [24]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [25]:
model.fc = nn.Linear(512, 70)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [26]:
len(list(model.children()))

10

In [27]:
list(model.children())[5]

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-

In [28]:
for i, layer in enumerate(model.children()):
    if i < 9:
        for param in layer.parameters():
            param.requires_grad = False

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [32]:
model = model.to(device)

In [33]:
loss_fn = torch.nn.CrossEntropyLoss()

learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [35]:
def evaluate(model, dataloader, loss_fn):
    losses = []
    
    num_correct = 0
    num_elements = len(dataloader)
    
    for i, batch in enumerate(dataloader):
        X_batch, y_batch = batch
        
        with torch.no_grad():
            X_batch = X_batch.to(device)
            logits = model(X_batch)
            
            y_batch = y_batch.to(device)
            loss = loss_fn(logits, y_batch)
            losses.append(loss.item())
            
            y_pred = torch.argmax(logits, dim=1)
            num_correct += torch.sum(y_pred == x_batch)
            
    accuracy = num_correct / num_elements
    
    return accuracy, np.mean(losses)

In [36]:
def train(model, loss_fn, optimizer, n_epoch=3):    
    for epoch in range(n_epoch):
        
        print(f"Epoch: {epoch}")
        model.train(True)
        
        for i, batch in enumerate(train_loader):
            X_batch, y_batch = batch
            
            X_batch = X_batch.to(device)
            logits = model(X_batch)
            
            y_batch = y_batch.to(device)
            loss = loss_fn(logits, y_batch)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        model.train(False)
        
        val_accuracy, val_loss = evaluate(model, val_loader, loss_fn=loss_fn)
        
        print(f"Loss/val {val_loss.item()}")
        print(f"Accuracy/val {val_accuracy}")
        
    return model

In [ ]:
model = train(model, loss_fn, optimizer, n_epoch=3)

Epoch: 0


In [ ]:
train_accuracy, _ = evaluate(model, optimizer, loss_fn)
print(f"Train accuracy: {train_accuracy}")

test_accuracy, _ = evaluate(model, optimizer, loss_fn)
print(f"Test accuracy: {test_accuracy}")

In [ ]:
torch.save(model, "/kaggle/working/model_dogs.pt")

In [ ]:
model_new = torch.load("/kaggle/working/model_dogs.pt")

test_accuracy, _ = evaluate(model, test_loader, loss_fn)
print(f"Test accuracy: {test_accuracy}")